In [121]:
import numpy as np
import matplotlib.image as mpimg
import re
import cv2
import os
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as data
import torch.nn as nn

In [122]:
train_transforms = transforms.Compose([transforms.Resize((224,224))])

In [148]:
train_data_images = torchvision.datasets.ImageFolder(root = 'training/images', transform = train_transforms, loader = torchvision.io.read_image)
train_data_groundtruth = torchvision.datasets.ImageFolder(root = 'training/groundtruth', transform = train_transforms, loader = torchvision.io.read_image)

In [124]:
# Red pixel value
print(train_data_images[0][0][0].size())
# Green pixel value
print(train_data_images[0][0][1].size())
# Blue pixel value
print(train_data_images[0][0][2].size())
#length train_data_images
print(len(train_data_images))

torch.Size([224, 224])
torch.Size([224, 224])
torch.Size([224, 224])
100


In [83]:
train_loader_images = torch.utils.data.DataLoader(train_data_images,batch_size=30)
train_loader_groundtruth = torch.utils.data.DataLoader(train_data_groundtruth,batch_size=30)

In [107]:
print(train_loader_images.getitems())

AttributeError: 'DataLoader' object has no attribute 'getitems'

In [125]:
def set_device():
    if torch.cuda.is_available():
        dev ="cuda:0"
    else:
        dev = "cpu"
    return torch.device(dev)

In [183]:
class Model(nn.Module):
    # Models in PyTorch usually inherit from this Module
    def __init__(self):
        super().__init__()

        self.input_layer = torch.nn.Linear(50176, 128)
        self.input_phi = torch.nn.ReLU()
        self.layer1 = torch.nn.Linear(128, 128)
        self.phi1 = torch.nn.ReLU()
        self.layer2 = torch.nn.Linear(128, 128)
        self.phi2 = torch.nn.ReLU()
        self.output_layer = torch.nn.Linear(128, 10)
        
    def forward(self, Z):
        Z = torch.flatten(Z, 1)  # Flatten (n, 28, 28) to (n, 784)
        Z = Z.to(torch.float32)
        Z = self.input_layer(Z)
        Z = self.input_phi(Z)

        Z = self.layer1(Z)
        Z = self.phi1(Z)

        Z = self.layer2(Z)
        Z = self.phi2(Z)

        Z = self.output_layer(Z)

        return Z       

In [184]:
def train_epoch(model, device, train_data_images, train_data_groundtruth, optimizer, max_iter, criterion):
    model.train()
    
    loss_history = []
    accuracy_history = [] 
    batch = np.random.choice(np.arange(len(train_data_images)), size=30, replace=False)
    
    for i, batch_idx in enumerate (batch):        
        x = train_data_images[batch_idx][0]
        y = train_data_groundtruth[batch_idx][0]
        x = x.to(device)
        y = y.to(device)
        
        y_hat = model(x)       
        loss = criterion(y_hat, y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        pred = y_hat.argmax(dim=1, keepdim=True)
        correct = pred.eq(y.view_as(pred)).sum().item()

        loss_history.append(loss.item())
        accuracy_history.append(correct / len(x))

        if batch_idx % (len(train_data_images.dataset) // len(x) // 10) == 0:
            print(
                f"Train Epoch: {epoch}-{batch_idx} batch_loss={loss.item()/len(x):0.2e} batch_acc={correct/len(x):0.3f}"
            )

    return loss_history, accuracy_history
        
        

In [185]:
def run_mnist_training(num_epochs, lr, device):
    
    # ===== Model, Optimizer and Criterion =====
    model = Model()
    model = model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = torch.nn.functional.cross_entropy
    
    # ===== Train Model =====
    train_loss_history = []
    train_acc_history = []
    for epoch in range(1, num_epochs + 1):
        train_loss, train_acc = train_epoch(
            model, device, train_data_images, train_data_groundtruth, optimizer, epoch, criterion)
        train_loss_history.extend(train_loss)
        train_acc_history.extend(train_acc)
        
    # ===== Plot training curves =====
    n_train = len(train_acc_history)
    t_train = num_epochs * np.arange(n_train) / n_train
    t_val = np.arange(1, num_epochs + 1)
    plt.figure()
    plt.plot(t_train, train_acc_history, label="Train")
    plt.legend()
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")

    plt.figure()
    plt.plot(t_train, train_loss_history, label="Train")
    plt.legend()
    plt.xlabel("Epoch")
    plt.ylabel("Loss")

In [186]:
lr = 5e-3
num_epochs = 2
device = set_device()

run_mnist_training(num_epochs, lr, device)

ValueError: Expected input batch_size (3) to match target batch_size (1).